In [2]:
import numpy as np
import pandas as pd
import networkx as nx

import d3m
#from d3m.container import pandas as pd
from d3m.container.dataset import D3MDatasetLoader
from d3m.metadata import hyperparams

import os

import jhu_primitives as jhu
from jhu_primitives.utils import file_path_conversion
from jhu_primitives.utils.util import data_file_uri

# abs_file_path = os.path.abspath(os.getcwd())

# dataset_uri_TRUTH = data_file_uri(abs_file_path, uri = "file", datasetDoc = True, dataset_type = "")
# data_TRUTH = D3MDatasetLoader().load(dataset_uri = dataset_uri_TRUTH)

# dataset_uri_TRAIN = data_file_uri(abs_file_path, uri = "file", datasetDoc = True, dataset_type = "TRAIN")
# data_TRAIN = D3MDatasetLoader().load(dataset_uri = dataset_uri_TRAIN)

# dataset_uri_TEST = data_file_uri(abs_file_path, uri = "file", datasetDoc = True, dataset_type = "TEST")
# data_TEST = D3MDatasetLoader().load(dataset_uri = dataset_uri_TEST)

## DS01876

In [5]:
import _pickle as pickle

# pickle.dump(data_TRUTH, open('DS_truth.pkl', 'wb'))
# pickle.dump(data_TRAIN, open('DS_train.pkl', 'wb'))
# pickle.dump(data_TEST, open('DS_test.pkl', 'wb'))

truth = pickle.load(open('DS_truth.pkl', 'rb'))
train = pickle.load(open('DS_train.pkl', 'rb'))
test = pickle.load(open('DS_test.pkl', 'rb'))

In [ ]:
# ASE

In [ ]:
hp_lcc = jhu.lcc.lcc.Hyperparams().defaults
hp_ase = jhu.ase.ase.Hyperparams({'use_attributes': True, 'max_dimension': 5, 'which_elbow': 1})
hp_gmm = jhu.gclust.gclust.Hyperparams({'max_clusters': 10})

# Initialize
LCC = jhu.LargestConnectedComponent(hyperparams=hp_lcc)
ASE = jhu.AdjacencySpectralEmbedding(hyperparams=hp_ase)
GMM = jhu.GaussianClustering(hyperparams=hp_gmm)

# Train
lcc_train = LCC.produce(inputs = train).value
ase_train = ASE.produce(inputs = lcc_train).value
GMM.set_training_data(inputs = ase_train)
GMM.fit()

# Test
lcc_test = LCC.produce(inputs = test).value
ase_test = ASE.produce(inputs = lcc_test).value
predictions = GMM.produce(inputs = ase_test).value

truth_labels = np.array(truth['learningData']['classLabel'])[np.array(predictions['d3mIndex']).astype(int) - 1]
preds = np.array(predictions['classLabel'])
np.sum(preds.astype(str) == truth_labels)/len(preds)

In [ ]:
# LSE

In [24]:
hp_lcc = jhu.lcc.lcc.Hyperparams().defaults
hp_lse = jhu.lse.lse.Hyperparams({'use_attributes': True, 'max_dimension': 5, 'which_elbow': 1})
hp_gmm = jhu.gclust.gclust.Hyperparams({'max_clusters': 10})

# Initialize
LCC = jhu.LargestConnectedComponent(hyperparams=hp_lcc)
LSE = jhu.LaplacianSpectralEmbedding(hyperparams=hp_lse)
GMM = jhu.GaussianClustering(hyperparams=hp_gmm)

# Train
lcc_train = LCC.produce(inputs = train).value
lse_train = LSE.produce(inputs = lcc_train).value
GMM.set_training_data(inputs = ase_train)
GMM.fit()

# Test
lcc_test = LCC.produce(inputs = test).value
lse_test = LSE.produce(inputs = lcc_test).value
predictions = GMM.produce(inputs = lse_test).value

truth_labels = np.array(truth['learningData']['classLabel'])[np.array(predictions['d3mIndex']).astype(int) - 1]
preds = np.array(predictions['classLabel'])
np.sum(preds.astype(str) == truth_labels)/len(preds)

0.01137009664582149

In [25]:
# SGC

In [27]:
hp_sgc = jhu.sgc.sgc.Hyperparams().defaults()

SGC = jhu.SpectralGraphClustering(hyperparams=hp_sgc)

SGC.set_training_data(inputs = train)
SGC.fit()
predictions = SGC.produce(inputs = test).value

truth_labels = np.array(truth['learningData']['classLabel'])[np.array(predictions['d3mIndex']).astype(int) - 1]
preds = np.array(predictions['classLabel'])
np.sum(preds.astype(str) == truth_labels)/len(preds)

0.050028425241614556